In [51]:
%pip install kagglehub numpy pandas seaborn matplotlib scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [52]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [53]:
dataset_path = "./Breast_Cancer.csv"

df = pd.read_csv(dataset_path)

df.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


In [54]:
df = df.drop(columns=["Grade"])

In [55]:
# Create a new column "Regional Node Positive" / "Regional Node Examined" ratio
# df["Node Positive Ratio"] = df["Reginol Node Positive"] / df["Regional Node Examined"]
# df.drop(columns=["Reginol Node Positive", "Regional Node Examined"])

# Rename column "A Stage" to "M Stage"
df = df.rename(columns={"A Stage": "M Stage"})

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
import pandas as pd

# 0 -> T1, 1 -> T2, 2 -> T3, 3 -> T4
# t_stage = df["T Stage "].unique().tolist()
t_stage_order = ["T1", "T2", "T3", "T4"]
# print(t_stage)
t_stage_ordinal_encoder = OrdinalEncoder(categories=[t_stage_order])
df["T Stage Encoded"] = t_stage_ordinal_encoder.fit_transform(df[["T Stage "]])

# 0 -> Distant, 1 -> Regional
df["M Stage Encoded"] = LabelEncoder().fit_transform(df["M Stage"])

# 0 -> N1, 1 -> N2, 2 -> N3
n_stage = ["N1", "N2", "N3"]
n_stage_ordinal_encoder = OrdinalEncoder(categories=[n_stage])
df["N Stage Encoded"] = n_stage_ordinal_encoder.fit_transform(df[["N Stage"]])

# 0 -> Negative, 1 -> Positive
df["Estrogen Status Encoded"] = LabelEncoder().fit_transform(df["Estrogen Status"])
df["Progesterone Status Encoded"] = LabelEncoder().fit_transform(
    df["Progesterone Status"]
)

df = pd.get_dummies(df, columns=["Race"], prefix="Race")
race_cols = [col for col in df.columns if col.startswith("Race_")]
df[race_cols] = df[race_cols].astype(int)

# stage_order = ["IIA", "IIB", "IIIA", "IIIB", "IIIC"]

# encoder = OrdinalEncoder(categories=[stage_order])
# df["6th_Stage_Encoded"] = encoder.fit_transform(df[["6th Stage"]])


df["differentiate"] = (
    df["differentiate"]
    .replace(
        {
            "Well differentiated": 1,
            "Moderately differentiated": 2,
            "Poorly differentiated": 3,
            "Undifferentiated": 4,
        }
    )
    .astype(int)
)

df = pd.get_dummies(df, columns=["Marital Status"], prefix="Marital")
marital_cols = [col for col in df.columns if col.startswith("Marital_")]
df[marital_cols] = df[marital_cols].astype(int)

le = LabelEncoder()
# Alive -> 0, Dead -> 1
df["Status_encoded"] = le.fit_transform(df["Status"])

/var/folders/bh/yxgmvcwn49390srvg5kl2d01f3vnyl/T/ipykernel_77187/2976349040.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(


In [57]:
df.head()

,Age,T Stage,N Stage,6th Stage,differentiate,M Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,...,Progesterone Status Encoded,Race_Black,Race_Other,Race_White,Marital_Divorced,Marital_Married,Marital_Separated,Marital_Single,Marital_Widowed,Status_encoded
0,68,T1,N1,IIA,3,Regional,4,Positive,Positive,24,...,1,0,0,1,0,1,0,0,0,0
1,50,T2,N2,IIIA,2,Regional,35,Positive,Positive,14,...,1,0,0,1,0,1,0,0,0,0
2,58,T3,N3,IIIC,2,Regional,63,Positive,Positive,14,...,1,0,0,1,1,0,0,0,0,0
3,58,T1,N1,IIA,3,Regional,18,Positive,Positive,2,...,1,0,0,1,0,1,0,0,0,0
4,47,T2,N1,IIB,3,Regional,41,Positive,Positive,3,...,1,0,0,1,0,1,0,0,0,0


In [58]:
df["Status"].value_counts()

Status
Alive    3408
Dead      616
Name: count, dtype: int64

In [59]:
df = df.drop(
    columns=[
        "T Stage ",
        "M Stage",
        "N Stage",
        "Estrogen Status",
        "Progesterone Status",
        "Status",
        "6th Stage",
        # "Race",
        # "Marital Status",
        # "Regional Node Examined",
        # "Reginol Node Positive",
    ]
)

In [60]:
df.head()

,Age,differentiate,Tumor Size,Regional Node Examined,Reginol Node Positive,Survival Months,T Stage Encoded,M Stage Encoded,N Stage Encoded,Estrogen Status Encoded,Progesterone Status Encoded,Race_Black,Race_Other,Race_White,Marital_Divorced,Marital_Married,Marital_Separated,Marital_Single,Marital_Widowed,Status_encoded
0,68,3,4,24,1,60,0.0,1,0.0,1,1,0,0,1,0,1,0,0,0,0
1,50,2,35,14,5,62,1.0,1,1.0,1,1,0,0,1,0,1,0,0,0,0
2,58,2,63,14,7,75,2.0,1,2.0,1,1,0,0,1,1,0,0,0,0,0
3,58,3,18,2,1,84,0.0,1,0.0,1,1,0,0,1,0,1,0,0,0,0
4,47,3,41,3,1,50,1.0,1,0.0,1,1,0,0,1,0,1,0,0,0,0


In [61]:
from sklearn.utils import resample


df_majority = df[df["Status_encoded"] == 0]
df_minority = df[df["Status_encoded"] == 1]

df_majority_downsampled = resample(
    df_majority,
    replace=False,
    n_samples=len(df_minority),  # ให้เท่ากับ minority
    random_state=42,
)

df_balanced = pd.concat([df_majority_downsampled, df_minority])
df_balanced.head()

,Age,differentiate,Tumor Size,Regional Node Examined,Reginol Node Positive,Survival Months,T Stage Encoded,M Stage Encoded,N Stage Encoded,Estrogen Status Encoded,Progesterone Status Encoded,Race_Black,Race_Other,Race_White,Marital_Divorced,Marital_Married,Marital_Separated,Marital_Single,Marital_Widowed,Status_encoded
34,46,2,30,19,10,31,1.0,1,2.0,1,1,0,1,0,0,1,0,0,0,0
4012,51,3,80,20,12,79,2.0,1,2.0,1,1,0,0,1,0,1,0,0,0,0
2813,62,3,20,1,1,60,0.0,1,0.0,1,1,0,0,1,0,0,0,1,0,0
1659,55,2,55,4,2,74,2.0,1,0.0,1,1,0,0,1,0,1,0,0,0,0
275,57,3,6,2,1,68,0.0,1,0.0,0,0,0,0,1,1,0,0,0,0,0


In [62]:
df_balanced.shape

(1232, 20)

In [63]:
df_balanced.to_csv("test_df_unbalanced_classification.csv", index=False)